Importing necessary libraries 

In [ ]:
# importing necessary libraries
import numpy as np
import pandas as pd
from datetime import datetime 
import calendar
from pandas_profiling import ProfileReport
import dtale
import sweetviz
import chart_studio.plotly as py
import seaborn as sns
import plotly
import plotly.express as px
from pandas_profiling import ProfileReport
%matplotlib 

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Section 1 : Data varying according to regions

In [ ]:
# gaining formatted data from the xls file
Data_region = pd.read_excel('87520072.xls', sheet_name= "Data_region_T")
Data_region_df = pd.DataFrame(Data_region)
Data_region_df.head(2)

In [ ]:
# renaming the columns of the data frame 
for col in Data_region_df.columns[3:83]:
    try:
        date = str(col.strftime("%b-%Y"))
    except AttributeError:
        continue
    Data_region_df.rename(columns={col:date}, inplace=True)
Data_region_df.head(2)

##### 1.3 saving files in csv and json format

In [ ]:
Data_region_df.to_csv("data_region_csv.csv")

In [ ]:
Data_region_df.to_json("data_region_json.json")

##### 1.4 - 1.5 Rows and Columns

In [ ]:
print('columns x rows : ',Data_region_df.size) 
print('no of rows, no of columns : ',Data_region_df.shape)  
print('no. of columns : ',len(Data_region_df.columns)) 
print('no. of rows    : ',len(Data_region_df.index)) 

------------------------------------------------------------------------------------------------------------------------------

In [ ]:
fig = px.scatter(Data_region_df_plots, x="Region", y=["Sep-2001","Dec-2001"])
fig.show()

In [ ]:
# applying the methodology to all the columns
year_range = range(1,22)
year_list = ["%02d" % x for x in year_range]

for i in range(len(year_list)):
    year='20'+year_list[i]
    Data_region_df[year] = Data_region_df[[ x for x in Data_region_df.columns if year in x]].sum(axis=1)
    i+=1

In [ ]:
# Data_region_df.columns.values
# Remove all columns between column name 'Sep-2001' to 'Jun-2021'
Data_year = Data_region_df.drop(Data_region_df.loc[:, 'Sep-2001':'Jun-2021'].columns, axis = 1)

In [ ]:
Data_year.head(2)

##### 1.6 Table

In [ ]:
column_names = ["Field Name", "Description", "Pandas Data Type","Data Scale","Min Value","Max Value","Unique Values","Missing Value Count"]
table = pd.DataFrame(columns = column_names)
#table

There are 4 types of scales of measurement namely nominal, ordinal, internal and ratio.   

In [ ]:
table['Field Name'] = Data_year.columns.values
table['Pandas Data Type'] = Data_year.dtypes.values
table['Data Scale'] = np.where(table['Pandas Data Type']=='object', 'Nominal', 'Ordinal')
table['Max Value'] = Data_year.max().values
table['Min Value'] = Data_year.min().values
table['Unique Values'] = Data_year.nunique().values
table['Missing Value Count'] = Data_year.isna().sum().values
table

##### 1.7 plotly charts

_1.7.1 scatter plot_

In [ ]:
fig1 = px.scatter(Data_year, x="Region", y=["2001","2002","2003"])
fig1.show()

In [ ]:
fig2 = px.scatter(Data_year, x="WorkType", y=["2001","2002"], color="Region", hover_data=['SeriesID'])
fig2.show()

_1.7.2 pie chart_

In [ ]:
pie_chart = px.pie(data_frame = Data_year,values = '2001',names = 'Region',color= 'Region',title='Residential Building Work in 2001 according to Region')
pie_chart

_1.7.3 bubble chart_

In [ ]:
melt_df = pd.melt(Data_year,id_vars=['SeriesID','Region','WorkType'],var_name='Year',value_name='Value')
melt_df.head()

In [ ]:
fig3 = px.scatter(melt_df, x="Year", y="Region",size="Value", color="Region", hover_name="WorkType", log_x=True, size_max=60)
fig3.show()

_1.7.4 bar chart_

In [ ]:
df_2001=melt_df[melt_df['Year']=='2001']
fig = px.bar(df_2001, x="Value", y="WorkType", color="Region", title="Bar Chart")
fig.show()

_1.7.5 Histogram_

In [ ]:
fig = px.histogram(df_2001, x="Value", color="Region")
fig.show()

_1.7.6 maps_

In [ ]:
import os
import folium
from folium import plugins
import ipywidgets
import geocoder
import geopy
from geopy.geocoders import Nominatim 

In [ ]:
import requests
import json

In [ ]:
Data_maps = Data_year.copy()

In [ ]:
Data_maps['latitude'] = np.nan
Data_maps['longitude'] = np.nan
Data_maps.head(2)

In [ ]:
Data_maps['Region'] = Data_maps['Region'].str.rstrip()
Data_maps['Region'] = Data_maps['Region'].str.lstrip()
# Data_maps['Region'] = Data_maps['Region'].str.replace(" ","")

In [ ]:
Data_maps['Region'].unique()

In [ ]:
Data_maps

In [ ]:
col_names = ["Region", "Latitude", "Longitude"]
map_df = pd.DataFrame(columns = col_names)
map_df

In [ ]:
places = Data_maps['Region'].unique()
#places = [x.rstrip(' ') for x in places]
places

In [ ]:
for i in places:
    parameters = {
        'key' : 'WUS5vAMI6fT412iMzgIe398Itfwvty06',
        'location' : i
    }
    response = requests.get('http://www.mapquestapi.com/geocoding/v1/address',params = parameters )
    #print(response.text)
    data = json.loads(response.text)['results']

    lat = data[0]['locations'][0]['latLng']['lat']
    lng = data[0]['locations'][0]['latLng']['lng']
    
    map_df.at[i,'Region'] = i
    map_df.at[i,'Latitude'] = lat
    map_df.at[i,'Longitude'] = lng

In [ ]:
map_df

In [ ]:
map1 = folium.Map(location=[-24.30891080919393, 134.79035784459924],zoom_start=3,width=500,height=300,control_scale=True)

folium.raster_layers.TileLayer('Open Street Map').add_to(map1)
folium.raster_layers.TileLayer('Stamen Terrain').add_to(map1)
folium.raster_layers.TileLayer('CartoDB Positron').add_to(map1)
folium.LayerControl().add_to(map1)
    
for (index, row) in map_df.iterrows():
    folium.Marker(location=[row.loc['Latitude'],row.loc['Longitude']],
                 popup = row.loc['Region'],
                 tooltip='click').add_to(map1)

map1

##### 1.8 Correlation

Pearson correlation
>The Pearson correlation evaluates the linear relationship between two continuous variables. A relationship is linear when a change in one variable is associated with a proportional change in the other variable.

In [ ]:
pearson_df = Data_year.loc[:,Data_year.columns[3:]]
pearson_corr=pearson_df.corr()
pearson_corr

In [ ]:
import plotly.figure_factory as ff
fig = px.imshow(pearson_corr)
fig.show()

Spearman Correlation
>The Spearman correlation evaluates the monotonic relationship between two continuous or ordinal variables. In a monotonic relationship, the variables tend to change together, but not necessarily at a constant rate. 

In [ ]:
sp_df = Data_year.loc[:,Data_year.columns[3:]]
spearman_corr=sp_df.corr(method="spearman")
spearman_corr

In [ ]:
fig = px.imshow(spearman_corr)
fig.show()

##### 1.9 Folium Maps

In [ ]:
Data_maps_folium = Data_maps.copy()
Data_maps_folium = Data_maps_folium.set_index('Region')
Data_maps_folium.head(2)

In [ ]:
map_df_folium = map_df.copy()
map_df_folium = map_df.set_index('Region')
map_df_folium = map_df_folium.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude'})
map_df_folium

In [ ]:
Data_maps_folium.update(map_df_folium)
Data_maps_folium.head(2)

In [ ]:
for i in places:
    parameters = {
        'key' : 'WUS5vAMI6fT412iMzgIe398Itfwvty06',
        'location' : i
    }
    response = requests.get('http://www.mapquestapi.com/geocoding/v1/address',params = parameters )
    #print(response.text)
    data = json.loads(response.text)['results']

    lat = data[0]['locations'][0]['latLng']['lat']
    lng = data[0]['locations'][0]['latLng']['lng']
    
    map_df.at[i,'Region'] = i
    map_df.at[i,'Latitude'] = lat
    map_df.at[i,'Longitude'] = lng

In [ ]:
x = Data_maps.groupby("Region").mean()
x.update(map_df_folium)

In [ ]:
x

In [ ]:
x['Region'] = x.index
x.reset_index(drop=True, inplace=True)
x.head(2)

In [ ]:
map2 = folium.Map(location=[-24.30891080919393, 134.79035784459924],zoom_start=3,width=500,height=300,control_scale=True)
borderStyle = {'color':'green','weight':1, 'fill':False}

folium.GeoJson(data='australian_states.geojson',
              name = 'borders',
              style_function = lambda x: borderStyle).add_to(map2)

stateLayer = folium.FeatureGroup('Australian States').add_to(map2)

for i in range(len(x)):
    latStr = x.iloc[i]['latitude']
    lngStr = x.iloc[i]['longitude']
    region = x.iloc[i]['Region']
    yr2001 = x.iloc[i]['2001']
    cRad = yr2001
    folium.Circle(
        location= [latStr,lngStr],
        radius = cRad,
        weight = 1,
        fill=True
    ).add_to(map2)

folium.LayerControl().add_to(map2)
map2

##### 1.10 Summary statistics

In [ ]:
Data_year.groupby("Region").mean()

In [ ]:
Data_year.describe()

In [ ]:
Data_year.groupby("WorkType")["2001"].count()

In [ ]:
Data_year.groupby("Region")["2001"].count()

In [ ]:
Data_year.groupby(["WorkType","Region"])['2001'].mean()

### Section 2 : Data Australia (Summary)

In [ ]:
# gaining formatted data from the xls file
Data_aus = pd.read_excel('87520072.xls', sheet_name= "Data_Aus_T")
Data_aus_df = pd.DataFrame(Data_aus)
Data_aus_df.head(2)

In [ ]:
# renaming the columns of the data frame 
for col in Data_aus_df.columns[2:82]:
    try:
        date = str(col.strftime("%b-%Y"))
    except AttributeError:
        continue
    Data_aus_df.rename(columns={col:date}, inplace=True)
Data_aus_df.head(2)

#### Pandas Profiling

In [ ]:
profile = ProfileReport(Data_aus_df,title='Pandas Profiling Report',html={'style':{'full_width':True}})

In [ ]:
#profile

##### Pandas dtale

In [ ]:
dtale.show(Data_aus_df)